In [1]:
library(future)
library(Seurat)
library(data.table)
library(dplyr)
library(ggplot2)
library(Matrix)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Warning message:
“package ‘data.table’ was built under R version 4.4.2”
Warning message:
“package ‘dplyr’ was built under R version 4.4.1”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘ggplot2’ was built under R version 4.4.1”
Warning message:
“package ‘Matrix’ was built under R version 4.4.3”


In [2]:
# With 800GB RAM available, we can be much more generous
RAM.thresh <- 200000 * 1024^2  # 200GB limit for future operations
options(future.globals.maxSize = RAM.thresh)

In [3]:
# Use all 12 CPUs since you have abundant memory
plan(multisession, workers = 12)

In [4]:
# Load object
cat("Loading Seurat object...\n")
sce1 <- readRDS("/tscc/lustre/ddn/scratch/pkudtarkar/FNIH/250429_Reference_heart_preprocessed.rds")

Loading Seurat object...


In [5]:
# Set identity
Idents(sce1) <- "celltype"
sce1$Cell_Type <- Idents(sce1)

cat("Dataset info:\n")
cat("Number of cells:", ncol(sce1), "\n")
cat("Number of genes:", nrow(sce1), "\n")
cat("Cell types:\n")
print(table(Idents(sce1)))

Loading required package: Signac



Dataset info:
Number of cells: 329255 
Number of genes: 36510 
Cell types:

       Fibroblast       Endothelial               vCM       Macrophages 
            67157             52039             68449             36401 
        Pericytes              Mast       Endocardial                 T 
            22489              1986              8927             14760 
               SM                DC                 B          Neuronal 
             6561             12968              1535              3962 
               NK lymph_Endothelial         Adipocyte        Epicardial 
             3726              1611              1122              1716 
              aCM 
            23846 


In [6]:
# Get normalized data
cat("Extracting normalized data...\n")
normalized_matrix <- GetAssayData(sce1, assay = "RNA", layer = "data")

Extracting normalized data...


In [7]:
cat("Sample values from normalized matrix:\n")
print(normalized_matrix[1:5, 1:5])

Sample values from normalized matrix:
5 x 5 sparse Matrix of class "dgCMatrix"
            QY_2193_1_2_QY_2192_1_2_AAACAGCCAACTAGGG-1
MIR1302-2HG                                          .
FAM138A                                              .
OR4F5                                                .
AL627309.1                                           .
AL627309.3                                           .
            QY_2193_1_2_QY_2192_1_2_AAACAGCCACTTACAG-1
MIR1302-2HG                                          .
FAM138A                                              .
OR4F5                                                .
AL627309.1                                           .
AL627309.3                                           .
            QY_2193_1_2_QY_2192_1_2_AAACAGCCAGTTTGTG-1
MIR1302-2HG                                          .
FAM138A                                              .
OR4F5                                                .
AL627309.1                               

In [8]:
cat("Estimated dense matrix size: ~450GB\n")
cat("Available RAM: 800GB\n")
cat("Using chunked processing for safety...\n")

Estimated dense matrix size: ~450GB
Available RAM: 800GB
Using chunked processing for safety...


In [9]:
# Process by gene chunks to stay within memory limits
cat("Starting chunked dense matrix export...\n")
output_file <- "/tscc/lustre/ddn/scratch/pkudtarkar/FNIH_scRNA_log_normalized_gene_by_cell_matrix-heart.tsv"

# Conservative chunk size for 1.5M cells
chunk_size <- 500  # Process 500 genes at a time (~30GB per chunk)

n_genes <- nrow(normalized_matrix)
n_chunks <- ceiling(n_genes / chunk_size)

cat("Processing", n_genes, "genes in", n_chunks, "chunks of", chunk_size, "genes each\n")
cat("Each chunk will use approximately 30GB RAM\n")

Starting chunked dense matrix export...
Processing 36510 genes in 74 chunks of 500 genes each
Each chunk will use approximately 30GB RAM


In [10]:
# Write header first
cell_names <- colnames(normalized_matrix)
header <- c("Gene", cell_names)
cat("Writing header with", length(cell_names), "cell names...\n")
write.table(t(header), file = output_file, sep = "\t", quote = FALSE, 
            row.names = FALSE, col.names = FALSE)

Writing header with 329255 cell names...


In [11]:
# Process each chunk
start_time <- Sys.time()

for (i in 1:n_chunks) {
  chunk_start_time <- Sys.time()
  
  start_idx <- ((i-1) * chunk_size) + 1
  end_idx <- min(i * chunk_size, n_genes)
  
  cat("Processing chunk", i, "of", n_chunks, "(genes", start_idx, "to", end_idx, ")")
  
  # Extract chunk and convert to dense
  chunk_sparse <- normalized_matrix[start_idx:end_idx, , drop = FALSE]
  chunk_dense <- as.matrix(chunk_sparse)
  chunk_df <- as.data.frame(chunk_dense)
  chunk_dt <- as.data.table(chunk_df, keep.rownames = "Gene")
  
  # Append to file
  fwrite(chunk_dt, file = output_file, sep = "\t", quote = FALSE, 
         append = TRUE, col.names = FALSE)
  
  # Clean up chunk immediately
  rm(chunk_sparse, chunk_dense, chunk_df, chunk_dt)
  gc()
  
  # Time estimation
  chunk_end_time <- Sys.time()
  chunk_duration <- as.numeric(difftime(chunk_end_time, chunk_start_time, units = "mins"))
  
  cat(" - completed in", round(chunk_duration, 2), "minutes\n")
  
  # Progress and time estimation
  if (i %% 10 == 0 || i == n_chunks) {
    elapsed_time <- as.numeric(difftime(Sys.time(), start_time, units = "mins"))
    avg_time_per_chunk <- elapsed_time / i
    remaining_chunks <- n_chunks - i
    estimated_remaining <- remaining_chunks * avg_time_per_chunk
    
    cat("Progress: ", round(100 * i / n_chunks, 1), "% complete\n")
    cat("Elapsed time: ", round(elapsed_time, 1), " minutes\n")
    cat("Estimated remaining time: ", round(estimated_remaining, 1), " minutes\n")
    cat("Estimated total time: ", round(elapsed_time + estimated_remaining, 1), " minutes\n\n")
  }
}

total_time <- as.numeric(difftime(Sys.time(), start_time, units = "mins"))
cat("Dense matrix export completed in", round(total_time, 1), "minutes!\n")

# Clean up the large normalized matrix
rm(normalized_matrix)
gc()


Processing chunk 1 of 74 (genes 1 to 500 )

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.2 GiB"


 - completed in 0.22 minutes
Processing chunk 2 of 74 (genes 501 to 1000 )

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.2 GiB"


 - completed in 0.2 minutes
Processing chunk 3 of 74 (genes 1001 to 1500 )

...

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.2 GiB"


 - completed in 0.2 minutes
Processing chunk 73 of 74 (genes 36001 to 36500 )

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.2 GiB"


 - completed in 0.19 minutes
Processing chunk 74 of 74 (genes 36501 to 36510 ) - completed in 0.12 minutes
Progress:  100 % complete
Elapsed time:  14.5  minutes
Estimated remaining time:  0  minutes
Estimated total time:  14.5  minutes

Dense matrix export completed in 14.5 minutes!


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,14519122,775.5,28915812,1544.3,28915812,1544.3
Vcells,9913428722,75633.5,16654213030,127061.6,11009850188,83998.5


In [13]:
sce1

An object of class Seurat 
392885 features across 329255 samples within 4 assays 
Active assay: RNA (36510 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 3 other assays present: RNA_raw, SCT, ATAC
 10 dimensional reductions calculated: pca, harmony.rna, umap.rna, lsi, harmony.atac, umap.atac, umap.wnn, joint.umap, spca, umap.harmony.rna

In [14]:
# Extract and save UMAP coordinates
cat("Saving UMAP coordinates...\n")
umap_coordinates <- Embeddings(sce1, "umap.rna")
colnames(umap_coordinates) <- c("X", "Y")
write.table(umap_coordinates, 
           file = "/tscc/lustre/ddn/scratch/pkudtarkar/umap-heart.tsv", 
           sep = "\t", quote = FALSE, row.names = TRUE, col.names = TRUE)

Saving UMAP coordinates...


In [17]:
# Extract and save metadata
cat("Saving cell metadata...\n")
cell_metadata <- sce1@meta.data
write.table(cell_metadata, 
           file = "/tscc/lustre/ddn/scratch/pkudtarkar/metadata-heart.tsv", 
           sep = "\t", quote = FALSE, row.names = TRUE, col.names = TRUE)


Saving cell metadata...


In [16]:
# Verify outputs
cat("Verification:\n")
cat("Dense matrix file size:\n")
system(paste("ls -lh", output_file))

cat("First 3 rows and 5 columns of output:\n")
verification <- fread(output_file, nrows = 3, select = 1:min(6, 1528726))
print(verification[, 1:min(5, ncol(verification))])

cat("All processing completed successfully!\n")
cat("Output files:\n")
cat("1. Dense matrix:", output_file, "\n")
cat("2. UMAP coordinates: /tscc/lustre/ddn/scratch/pkudtarkar/umap-heart.tsv\n")
cat("3. Cell metadata: /tscc/lustre/ddn/scratch/pkudtarkar/metadata-heart.tsv\n")

cat("\nFinal summary:\n")
cat("- Dataset: 38,531 genes × 1,528,725 cells\n")
cat("- Processing time:", round(total_time, 1), "minutes\n")
cat("- Peak memory per chunk: ~30GB\n")
cat("- Total chunks processed:", n_chunks, "\n")

Verification:
Dense matrix file size:
First 3 rows and 5 columns of output:
          Gene QY_2193_1_2_QY_2192_1_2_AAACAGCCAACTAGGG-1
        <char>                                      <int>
1: MIR1302-2HG                                          0
2:     FAM138A                                          0
3:       OR4F5                                          0
   QY_2193_1_2_QY_2192_1_2_AAACAGCCACTTACAG-1
                                        <int>
1:                                          0
2:                                          0
3:                                          0
   QY_2193_1_2_QY_2192_1_2_AAACAGCCAGTTTGTG-1
                                        <int>
1:                                          0
2:                                          0
3:                                          0
   QY_2193_1_2_QY_2192_1_2_AAACAGCCATCCGTAA-1
                                        <int>
1:                                          0
2:                                  